In [12]:
pip install accelerate>=0.26.0

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import torch
from transformers import set_seed
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import json
import argparse
import random
import pickle
from tqdm import tqdm
import numpy as np
from ast import literal_eval
import re
import pandas as pd
import ast

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from transformers import DistilBertModel, DistilBertForMaskedLM,DistilBertTokenizer,DistilBertForTokenClassification

import string

c:\Users\Akhil\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from huggingface_hub import login
HF_HUB_DISABLE_IMPLICIT_TOKEN=1
login(token="")

In [ ]:
#save entropy and triplets 
path1=r""

In [ ]:
import pickle

# Open the pickle file in read-binary mode
with open(r'factkg\factkg_test.pickle', 'rb') as file:
    # Load the object from the file
    your_object = pickle.load(file)

In [3]:
data={'foundingYear': ['"2006"'], 'type': ['Private_company_limited_by_shares'], 'locationCity': ['Warsaw'], 
        'locationCountry': ['Poland'], 'regionServed': ['Poland'], 'keyPerson': ['Adam_Kuriański', 'Andrzej_Chajec'],
        'industry': ['"Telecommunications"', 'Telecommunication'], 'service': ['Telecommunication'], 'successor': ['"Aero 2"'], 
        'keyPeople': ['Adam_Kuriański', '"\'\'\'"', 'Andrzej_Chajec']}

In [4]:
query="I had heard that Mobyland has a successor!"

In [15]:
device="cpu"

In [42]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", device_map = 'auto')
model = AutoModel.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", device_map = 'auto')

Some parameters are on the meta device because they were offloaded to the disk and cpu.


In [43]:
def ans_cross_entroy(question, facts):
    input_text = ""
    for fact in facts:
        if input_text == "":
            input_text =  fact
        else:
            input_text =  input_text  + ', ' + fact 
    input_text = "Given fact: " + input_text + ', ' + question + '\nAnswer:'
    prom_text = input_text
    
    with torch.no_grad():
        input_ids = tokenizer.encode(question, return_tensors="pt").to(device)
        beam0_logits = model(input_ids).logits[:, -1, :].to(device)
        question_prob = torch.softmax(beam0_logits, dim=-1).to(device)

    with torch.no_grad():
        input_ids = tokenizer.encode(facts, return_tensors="pt").to(device)
        beam0_logits = model(input_ids).logits[:, -1, :].to(device)
        fact_prob = torch.softmax(beam0_logits, dim=-1).to(device)
    
    
    ans_h = -(question_prob * torch.log(fact_prob)).sum().to(device)

    if len(facts)>3:
        ans_h=ans_h/2
    return ans_h 

In [44]:
def ans_entroy(question, facts):
    input_text = ""
    for fact in facts:
        if input_text == "":
            input_text =  fact
        else:
            input_text =  input_text  + ', ' + fact 
    input_text = "Given fact: " + input_text + ', ' + question + '\nAnswer:'
    prom_text = input_text
    
    with torch.no_grad():
        input_ids = tokenizer.encode(prom_text, return_tensors="pt").to(device)
        beam0_logits = model(input_ids).logits[:, -1, :].to(device)
        beam0_prob = torch.softmax(beam0_logits, dim=-1).to(device)
        ans_h = -(beam0_prob * torch.log(beam0_prob)).sum().to(device)
    if len(facts)>3:
        ans_h=ans_h/2
    return ans_h 

In [86]:
def ans_cosine_sim(question, facts):
    input_text = ""
    for fact in facts:
        if input_text == "":
            input_text =  fact
        else:
            input_text =  input_text  + ', ' + fact 
    input_text = "Given fact: " + input_text + ', ' + question + '\nAnswer:'
    prom_text = input_text
    
    with torch.no_grad():
        input_ids = tokenizer(question, return_tensors="pt")
        beam0_logits = model(**input_ids)
        question_prob = beam0_logits.last_hidden_state
    facts.replace("[","")
    facts.replace("]","")
    facts.replace(","," ")
    facts.replace("'","")
    with torch.no_grad():
        input_ids = tokenizer(facts, return_tensors="pt")
        beam0_logits = model(**input_ids)
        fact_prob = beam0_logits.last_hidden_state
    
    # print(question_prob.shape)
    # print(fact_prob.shape)

    cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

    ans_h =cos(question_prob,fact_prob[:,0:12,:])

    return torch.mean(ans_h)

    


In [84]:
n=1
subgraph=[]
ent=[]
for relation, object in data.items():
                if [str(relation)] not in subgraph:
                    subgraph.append([str('Mobyland'),str(relation),str(object[0]).strip("\"")])
                    ent_temp=ans_cosine_sim(str(query),str(subgraph[0]))
                    ent.append(ent_temp)
                    print(subgraph[0],ent_temp)
                    
                    subgraph=[]
                

torch.Size([1, 12, 2048])
torch.Size([1, 15, 2048])
['Mobyland', 'foundingYear', '2006'] tensor(0.1761)
torch.Size([1, 12, 2048])
torch.Size([1, 18, 2048])
['Mobyland', 'type', 'Private_company_limited_by_shares'] tensor(0.1676)
torch.Size([1, 12, 2048])
torch.Size([1, 14, 2048])
['Mobyland', 'locationCity', 'Warsaw'] tensor(0.1685)
torch.Size([1, 12, 2048])
torch.Size([1, 14, 2048])
['Mobyland', 'locationCountry', 'Poland'] tensor(0.1602)
torch.Size([1, 12, 2048])
torch.Size([1, 15, 2048])
['Mobyland', 'regionServed', 'Poland'] tensor(0.1663)
torch.Size([1, 12, 2048])
torch.Size([1, 17, 2048])
['Mobyland', 'keyPerson', 'Adam_Kuriański'] tensor(0.1933)
torch.Size([1, 12, 2048])
torch.Size([1, 13, 2048])
['Mobyland', 'industry', 'Telecommunications'] tensor(0.1875)
torch.Size([1, 12, 2048])
torch.Size([1, 13, 2048])
['Mobyland', 'service', 'Telecommunication'] tensor(0.1750)
torch.Size([1, 12, 2048])
torch.Size([1, 16, 2048])
['Mobyland', 'successor', 'Aero 2'] tensor(0.1980)
torch.Size

In [85]:
print(np.argsort(ent))

[3 4 1 2 7 0 6 5 9 8]


In [27]:
n=1
subgraph=[]
for relation, object in data.items():
                if [str(relation)] not in subgraph:
                    subgraph.append([str('Mobyland'),str(relation),str(object[0]).strip("\"")])
                    print(subgraph[0],ans_entroy(str(query),str(subgraph[0])))
                    subgraph=[]

['Mobyland', 'foundingYear', '2006'] tensor(2.4138)
['Mobyland', 'type', 'Private_company_limited_by_shares'] tensor(2.4057)
['Mobyland', 'locationCity', 'Warsaw'] tensor(2.4802)
['Mobyland', 'locationCountry', 'Poland'] tensor(2.3519)
['Mobyland', 'regionServed', 'Poland'] tensor(2.3324)
['Mobyland', 'keyPerson', 'Adam_Kuriański'] tensor(2.4788)
['Mobyland', 'industry', 'Telecommunications'] tensor(2.3600)
['Mobyland', 'service', 'Telecommunication'] tensor(2.2837)
['Mobyland', 'successor', 'Aero 2'] tensor(2.4481)
['Mobyland', 'keyPeople', 'Adam_Kuriański'] tensor(2.4872)


In [ ]:

with open('dbpedia_2015_undirected_light.pickle', 'rb') as f:
    kg = pickle.load(f)
    


In [ ]:

from tqdm import tqdm
import pickle

qu = list(dataset.keys())
questions_dict={}
entity_set_dict = {}
label_set_dict = {}
q_id=0
for line in dataset:
    if not line:
        continue
    questions_dict[q_id]=qu[q_id]    
    entity_set_dict[q_id] = dataset[line]["Entity_set"]
    label_set_dict[q_id] = dataset[line]["Label"]
    q_id+=1


dataset_len=len(questions_dict)
a=0
k1=4
k2=4
data_num=range(a,dataset_len+1,1)
total_correct=0        
        
question_id_list=[]
question_list=[]
entity_set_list=[]
ground_truth_list=[]
contexts_list=[]

In [ ]:
def calc_ent(question,subgraph,n):
    ent=[]
    question=(str(question))
  
    for i in subgraph:
        ent.append(([i,ans_entroy(question,i).cpu().item()]))

    q=open('ent_2\\'+str(n)+'.txt','w',encoding='utf-8')
    q.write(str(question)+"\n"+str((ent)))
    q.close()


In [ ]:
n=0
for ii in tqdm(data_num):
    question = questions_dict[ii]
    entity_set = entity_set_dict[ii]
    ground_truth =label_set_dict[ii]
    
    # print(question, entity_set, ground_truth)
    # I have heard that Mobyland had a successor. ['Mobyland'] [True]
    
    triplets=[]
    cossim=[]
    i=0
    pos_list=[]
    # question_input = tokenizer([question], return_tensors="pt", padding=True, truncation=True).to(device)
    # question_embedding = question_model(**question_input).last_hidden_state.mean(dim=1)
    # question_embedding2 = question_model2(**question_input).last_hidden_state.mean(dim=1)
    
    subgraph = []

    for entity in entity_set:
        if entity in kg:
            
     
            # {'foundingYear': ['"2006"'], 'type': ['Private_company_limited_by_shares'], 'locationCity': ['Warsaw'], 
            # 'locationCountry': ['Poland'], 'regionServed': ['Poland'], 'keyPerson': ['Adam_Kuriański', 'Andrzej_Chajec'],
            # 'industry': ['"Telecommunications"', 'Telecommunication'], 'service': ['Telecommunication'], 'successor': ['"Aero 2"'], 
            # 'keyPeople': ['Adam_Kuriański', '"\'\'\'"', 'Andrzej_Chajec']}
            
            for relation, object in kg[entity].items():
                if [str(relation)] not in subgraph:
                    subgraph.append([str(entity),str(relation),str(object[0]).strip("\"")])
                
            #     m=0
            #     # print(kg[entity][relation])
            #     # ['"2006"']
            #     # ['Private_company_limited_by_shares']
            #     # ['Warsaw']
            #     # ['Poland']
            #     # ['Poland']
            #     # ['Adam_Kuriański', 'Andrzej_Chajec']
            #     # ['"Telecommunications"', 'Telecommunication']
            #     # ['Telecommunication']
            #     # ['"Aero 2"']
            #     # ['Adam_Kuriański', '"\'\'\'"', 'Andrzej_Chajec']
                
            #     # print(kg[entity][relation])
            #     # 2006
            calc_ent(question,subgraph,n)
            subgraph = []
    n+=1
            #     for obj in kg[entity][relation]:
            #         print(kg[obj].items())
            #         break 
                # break
        
    

In [ ]:
import os


files = os.listdir(path1)


In [ ]:

def ret1(q):
    arr1=[]
    arr0=[]

    data = ast.literal_eval(q)
    for i in data:
        arr0.append(i[0])
        arr1.append(i[1])
    srt=(np.argsort(arr1))
    arr=[arr0[i] for i in srt]
    return arr[0]    

In [ ]:

arr_z=[]
for i in files:
    t1=[]
    q=open(path1+"\\"+i,encoding="utf-8")
    print(i)
    temp=q.read()
    temp=temp.split("\n")
    t2=ret1(temp[1])
    arr_z.append([temp[0],t2])

In [ ]:
df = pd.DataFrame(arr_z, columns=['Attributes', 'Value'])
df.to_csv('ans_2.csv', index=False)

In [ ]:

s2=pd.read_csv("ans_2.csv")

In [ ]:
# prompt="""Verify the  following claims.\n The context is the evidence of triplets may help your verifying.\n
#             Each context contains triplets in the form of [head, relation, tail] and it means "head's relation is tail.".
#             Choose one of {True, False}:
#         """

# prompt+="""
#         Context 1: [['Ahamad_Kadhim', 'clubs', "Al-Zawra'a SC"], ] Claim 1: Ahmad Kadhim Assad's club is Al-Zawra'a SC.
#         Answer 1: True, based on the evidence set, Ahmad Kadhim Assad's club is Al-Zawra'a SC.
#         Context 2: [['Bananaman', 'firstAired', '"1983-10-03"'], ['Bananaman', 'starring', 'Tim_Brooke-Taylor'], ] Claim 2: Yeah! I know that a TV show, which starred Tim Brooke-Taylor, first aired on 3rd October 1983!
#         Answer 2: True, the claim is supported by the evidence since Bananaman refers to the TV show.
#         Context 3: [['Jamie_Lawrence', 'composer', 'Death_on_a_Factory_Farm'], ['Death_on_a_Factory_Farm', 'director', 'Sarah_Teale'], ] Claim 3: Really? Jamie Lawrence is the music composer of the 83 minute 'Death on a Factory Farm' film, directed by Sarah Teale!
#         Answer 3: False, there is no evidence for the 83 minute length.
#         Context 4: [[], ] Claim 4: Do you know Milan Hodža? he had a religion.
#         Answer 4: False, there is no evidence that Milan had a religion.
#         Context 5: [[], ] Claim 5: No, but the leader of the United States is not Olena Serdiuk.
#         Answer 5: True, based on the evidence set, there is no information that the leader of the United States is Olena Serdiuk.
#         Context 6: [['Brandon_Carter', 'almaMater', 'University_of_Cambridge'], ['Brandon_Carter', 'birthPlace', 'England'], ['University_of_Cambridge', 'viceChancellor', 'Leszek_Borysiewicz'], ] Claim 6: Brandon Carter was born in England and graduated from the University of Cambridge where the current Chancellor is Leszek Borysiewicz.
#         Answer 6: True, everything of the claim is supported by the evidence set.
#         Context 7: [['Unpublished_Story', 'director', 'Harold_French'], ['Unpublished_Story', 'cinematography', 'Bernard_Knowles'], ] Claim 7: 'A film' was produced by Anatole de Grunwald, directed by Harold French, with cinematography done by Bernard Knowles.
#         Answer 7: False, there is no information about the producer of 'Unpublished_Story'.
#         Context 8: [['200_Public_Square', 'location', 'Cleveland'], ['200_Public_Square', 'floorCount', '"45"'], ['Cleveland', 'country', 'United_States'], ] Claim 8: Yes, with a floor count of 45, 200 Public Square is located in Cleveland in the United States.
#         Answer 8: True, everything of the claim is supported by the evidence set.\n"""
#         #Context 9: [['Bananaman', 'starring', 'Bill_Oddie'], ['Bananaman', 'network', 'Broadcasting_House'], ['Bananaman', 'locationCity', 'Broadcasting_House'], ] Claim 9: Bananaman the TV series starred by a person was shown on the company and the company headquarters is called Broadcasting House.
#         #Answer 9: True, everything of the claim is supported by the evidence set.
#         #Context 10: [['Azerbaijan', 'leaderName', 'Artur_Rasizade'], ["Baku_Turkish_Martyrs'_Memorial", 'designer', '"Hüseyin Bütüner and Hilmi Güner"'], ["Baku_Turkish_Martyrs'_Memorial", 'location', 'Azerbaijan'], ] Claim 10: The place, designed by Huseyin Butuner and Hilmi Guner, is located in a country, where the leader is Artur Rasizade.
#         #Answer 10: True, everything of the claim is supported by the evidence set.
#         #Context 11: [['AIDAstella', 'shipBuilder', 'Meyer_Werft'], ['AIDAstella', 'shipOperator', 'AIDA_Cruises'], ] Claim 11: AIDA Cruise line operated the ship which was built by Meyer Werft in Townsend, Poulshot, Wiltshire.
#         #Answer 11: False, there is no evidence for Townsend, Poulshot, Wiltshire.
#         #Context 12: [[], ] Claim 12: An academic journal with code IJPHDE is also Acta Math. Hungar.
#         #Answer 12: False, there is no evidence that the academic journal is also Acta Math. Hungar.


# prompt+="""Now verify the following claim in the same way of these examples \n """


In [ ]:
# prompt = """Please verify the statement based on the given evidences from a knowledge graph. 

# Notes)

# 1). If there is any evidence that completely supports the statement, the answer is 'True', otherwise is 'False'.
# 2). Be careful for the relations in evidences. For example, 'builder' does not equal to 'owner', 'manager' does not equal to 'leader'.
# 3). For questions like 'A has a wife', if there is any evidence that A has a spouse with any name, the answer is 'True'.
# 4). When no evidence is provided, you MUST verify the statement and return 'True' or 'False' based on your common sense. NEVER say ANYTHING like 'Please provide more evidences'.
# 5). You should provide a brief reason with several words, then tell that the answer is 'True' or 'False'. NEVER say ANYTHING without direct reason like 'Based on the provided evidences, ...', 'Here is my reasoning: ...', etc.

# Examples)
# """
# prompt+="""
#         Context 1: [['Ahamad_Kadhim', 'clubs', "Al-Zawra'a SC"], ] Claim 1: Ahmad Kadhim Assad's club is Al-Zawra'a SC.
#         Answer 1: True, based on the evidence set, Ahmad Kadhim Assad's club is Al-Zawra'a SC.
#         Context 2: [['Bananaman', 'firstAired', '"1983-10-03"'], ['Bananaman', 'starring', 'Tim_Brooke-Taylor'], ] Claim 2: Yeah! I know that a TV show, which starred Tim Brooke-Taylor, first aired on 3rd October 1983!
#         Answer 2: True, the claim is supported by the evidence since Bananaman refers to the TV show.
#         Context 3: [['Jamie_Lawrence', 'composer', 'Death_on_a_Factory_Farm'], ['Death_on_a_Factory_Farm', 'director', 'Sarah_Teale'], ] Claim 3: Really? Jamie Lawrence is the music composer of the 83 minute 'Death on a Factory Farm' film, directed by Sarah Teale!
#         Answer 3: False, there is no evidence for the 83 minute length.
#         Context 4: [[], ] Claim 4: Do you know Milan Hodža? he had a religion.
#         Answer 4: False, there is no evidence that Milan had a religion.
        # """

# prompt+="""
#     Claim: Ahmad Kadhim Assad's club is Al-Zawra'a SC.
#     Answer: True.
#     Claim: Yeah! I know that a TV show, which starred Tim Brooke-Taylor, first aired on 3rd October 1983!
#     Answer: True.
#     Claim: Really? Jamie Lawrence is the music composer of the 83 minute 'Death on a Factory Farm' film, directed by Sarah Teale!
#     Answer: False.
#     Claim: Do you know Milan Hodža? he had a religion.
#     Answer: False.
#     Claim: No, but the leader of the United States is not Olena Serdiuk.
#     Answer: True.
#     """



# prompt = """Please verify the statement based on the given evidences from a knowledge graph. 

# Notes)

# 1). If there is any evidence that completely supports the statement, the answer is 'True', otherwise is 'False'.
# 2). For questions like 'A has a wife', if there is any evidence that A has a spouse with any name, the answer is 'True'.
# 2). When no evidence is provided, you MUST verify the statement and return 'True' or 'False' based on your common sense. NEVER say ANYTHING like 'Please provide more evidences'.
# 4). You should provide a brief reason with several words, then tell that the answer is 'True' or 'False'. NEVER say ANYTHING without direct reason like 'Based on the provided evidences, ...', 'Here is my reasoning: ...', etc.
# """

prompt=""" verify the following claim \n """

In [ ]:
x={}

In [ ]:

for i,j in zip(s2["Attributes"],s2["Value"]):

    temp_prompt=prompt
    temp_prompt+=" Claim: "+i+'\n'+'Answer: '
    chat = [
    {"role": "System", "content": "You are a helpful assistant that must absolutely return an answer from the set {True, False}."},
    {"role": "user", "content": temp_prompt},
            ]

    x[i]=tokenizer.apply_chat_template(chat, tokenize=True,add_generation_prompt=True, return_tensors="pt")

In [ ]:
n=1
for i in x:
    q=open('f_ans5\\'+str(n)+'.txt','w',encoding='utf-8')
    outputs = model.generate(torch.tensor(x[i]).clone().detach().to(device),temperature=0.2,top_p=0.1, max_new_tokens=400)
    q.write(str(tokenizer.decode(outputs[0])))
    q.close()
    print(n)
    n+=1

In [ ]:
l1=[]
n=0
for i in s2['Attributes'][1:]:
    if n==0:
        n=1
    q=open('f_ans4\\'+str(n)+'.txt','r',encoding='utf-8')
    n+=1
    e=q.readlines()
    l1.append(e[-1])

In [ ]:
n=0
labels=[]
ans=[]
for i in s2['Attributes'][1:]:
    c=(l1[n])
    c=c.lower().replace("<|eot_id|>","")
    c=''.join(e for e in c if e.isalnum() or e.isspace())
    c=c.split()
    print(c)
    if 'true' in (c):
        labels.append("True")
        ans.append([i,"True"])
    elif 'false' in (c):
        labels.append("False")  
        ans.append([i,"False"])
  
    else:
        labels.append("Non Answer")
        ans.append([i,"Non Answer"])

    n+=1


In [ ]:
n=0
acc=0
for i in s2['Attributes'][1:]:
    tl=your_object[i]["Label"]
    print(str(tl[0]))
    if str(tl[0])==labels[n]:
        acc+=1
    n+=1    